In [ ]:
import os
import time
import json

from utils.file_utils import *
from datasets import list_datasets
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from IPython import get_ipython
import pandas as pd
from tqdm import tqdm
from MergeModel import *
from ClassifierModel import *
import matplotlib.pyplot as plt

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', 2)

#### Bart tokenizer

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
vocab_size = tokenizer.vocab_size
vocab_size

### Classifier data

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("imdb")

In [ ]:
train_batch = dataset['train']['text']
train_labels = torch.tensor(dataset['train']['label'])

In [ ]:
val_batch = dataset['test']['text']
val_labels = torch.tensor(dataset['test']['label'])

In [ ]:
seq_length = 150
input_encoding = tokenizer(train_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)

In [ ]:
val_encoding = tokenizer(val_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)

In [ ]:
input_ids = input_encoding['input_ids']
input_mask = input_encoding['attention_mask']

In [ ]:
val_ids = val_encoding['input_ids']
val_mask = val_encoding['attention_mask']

In [ ]:
input_ids.shape, input_mask.shape, val_ids.shape, val_mask.shape

In [ ]:
# Creating DataLoaders

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# TRAINNG DATALOADER
batch_size = 16

train_data = TensorDataset(input_ids, input_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader_clf = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# # VALIDATION DATALOADER

# val_data = TensorDataset(val_ids, val_mask, val_labels)
# val_sampler = RandomSampler(val_data)
# val_dataloader_clf = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
# b = next(iter(train_dataloader_clf))

In [ ]:
# b[0].shape, b[1].shape, b[2].shape

##### end

#### BART Data

In [ ]:
df = pd.read_csv("../BART/test.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

In [ ]:
train_sentence = df.article.values
train_target = df.highlights.values

In [ ]:
# num_data_points = 10000
train_sentence = list(train_sentence)
train_target = list(train_target)

In [ ]:
article_encoding = tokenizer(train_sentence, return_tensors='pt', padding=True, truncation = True, max_length=500)
summary_encoding = tokenizer(train_target, return_tensors='pt', padding=True,truncation = True, max_length=100)

In [ ]:
article_input_ids = article_encoding['input_ids']
article_attention_mask = article_encoding['attention_mask']

In [ ]:
summary_input_ids = summary_encoding['input_ids']
summary_attention_mask = summary_encoding['attention_mask']

In [ ]:
print(article_input_ids.shape, article_attention_mask.shape)

In [ ]:
print(summary_input_ids.shape, summary_attention_mask.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4
train_data = TensorDataset(article_input_ids, article_attention_mask,\
                           summary_input_ids, summary_attention_mask)

In [ ]:
train_sampler = RandomSampler(train_data)

In [ ]:
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
def loss_fn(lm_logits, labels):
    loss_fct = CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
    loss = loss_fct(lm_logits.view(-1, vocab_size), labels.view(-1))
    return loss

In [ ]:
# mse_loss = nn.MSELoss()

In [ ]:
sent_model = ClassifierModel(vocab_size, 64, 2, 2, 512).cuda()

sent_model.load_state_dict(torch.load('classifier_model_senticlf.pt'))

In [ ]:
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
summary_model = summary_model.cuda()
senti_model = sent_model.cuda()

In [ ]:
# embedding_dim = 64
# out_dim = 2
# n_layers = 4
# hidden_size = 512
merge_model = MergeModel(summary_model, senti_model).cuda()
model_name = 'BART_classifier'
model_dir = './experiment'
model_path = os.path.join(model_dir,model_name)
epochs  = 100

In [ ]:
# for params in merge_model.sentiment_model.parameters():
#     params.requires_grad = False

In [ ]:
config = {
#     'embedding_dim': embedding_dim,
#     'out_dim': out_dim,
#     'n_layers': n_layers,
#     'hidden_size': hidden_size,
    'model_name': model_name,
    'epochs':epochs
}

In [ ]:
# Apply the weighting lambdas in the main function this is just a loss without lambda weights
def kl_div_loss(p_pred, p_target):
    
    softmax = nn.Softmax(dim=-1)
    
    logsoftmax = nn.LogSoftmax(dim=-1)
    
    kl_div = torch.nn.KLDivLoss()
    
    return kl_div(logsoftmax(p_pred),softmax(p_target))

In [ ]:
softmax = nn.Softmax(dim=-1)
cross_entropy = torch.nn.CrossEntropyLoss()

In [ ]:
def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False, model_dir='/experiment',config = None):
    
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    min_val_loss = np.inf
    
    model_name = config['model_name']
    print(f"Start training for Model {model_name}...\n")
    
    
#     if not os.path.exists(os.path.join(model_dir,model_name)):
#         os.mkdir(os.path.join(model_dir,model_name))
#     model_path = os.path.join(model_dir,model_name)
#     print(model_path)
#     write_to_file_in_dir(model_path, 'config.json', config)
    
#     train_log =  'train_log.txt'
#     write_string_train = f"Epoch, Train_Loss, Train_Acc"
#     log_to_file_in_dir(model_path, train_log, write_string_train)

#     if evaluation:
#         val_log = 'val_log.txt'
#         write_string_val = f"Epoch, Val_Loss, Val_Acc"
#         log_to_file_in_dir(model_path, val_log, write_string_val)
    
#     from transformers import AdamW
#     bart_optimizer = AdamW(model.parameters(), lr=1e-5)
    bart_optimizer = torch.optim.SGD(model.summary_model.parameters(),1e-5,momentum=0.9)
    
    sentiment_optimizer = torch.optim.Adam(model.sentiment_model.parameters())
    
    for epoch_i in range(epochs):
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Loss 1':^12} | {'Loss 2':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)
        t0_epoch, t0_batch = time.time(), time.time()

        total_loss, batch_loss, batch_loss1, batch_loss2, batch_loss_clf, batch_counts = 0, 0, 0, 0, 0, 0
        model.train()
        
        for step, batch in enumerate(train_dataloader):
            model.train()
            batch_counts +=1
            
            batch[0] = batch[0].cuda()
            batch[1] = batch[1].cuda()
            batch[2] = batch[2].cuda()
            batch[3] = batch[3].cuda()
            
            bart_optimizer.zero_grad()
            
            if epoch < 10:
                sentiment_optimizer.zero_grad()
                # get a random minibatch from the search queue with replacement
                batch_clf  = next(iter(train_dataloader_clf))
                input_clf  = batch_clf[0].cuda()
                target_clf = batch_clf[2].cuda()

                logits = model.sentiment_model(input_clf)
                loss_clf = cross_entropy(logits, target_clf)
            else:
                loss_clf = 0
            
            if epoch == 10:
                for params in merge_model.sentiment_model.parameters():
                    params.requires_grad = False
            
            summary_out,*sentiments = model(batch[0],batch[1], batch[2], batch[3])
        
            cost1, cost2 = 2, 1e-2
            
            loss1 = cost1*loss_fn(summary_out.logits, batch[2])
            
            loss2 = cost2*kl_div_loss(sentiments[0], sentiments[1])
            
            loss = loss1 + loss2 + loss_clf
            
            batch_loss += loss.item()
            
            batch_loss1 += loss1.item()
            
            batch_loss2 += loss2.item()
            
            total_loss += loss.item()
            
            train_losses.append(loss.item())
            
            # print(torch.autograd.grad(loss2, merge_model.summary_model.parameters(),retain_graph = True)[0])

            # write_string_train = f"{epoch_i}, {loss.item()}"
            # log_to_file_in_dir(model_path, train_log, write_string_train)
            
            loss1.backward(retain_graph = True)
            
            loss2.backward()
            
            torch.nn.utils.clip_grad_norm(model.parameters(), 5)
            
            bart_optimizer.step()
            
            if epoch < 10:
                sentiment_optimizer.step()
            
            if (step % 100 == 0) and (step != 0):
                time_elapsed = time.time() - t0_batch

                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {batch_loss1 / batch_counts:^12.6f} | {batch_loss2 / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                batch_loss, batch_loss1, batch_loss2, batch_counts = 0, 0, 0, 0
                t0_batch = time.time()
                
                print("-"*70)
                # print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_out.logits.argmax(dim = -1)])
        


        
        if ((epoch_i %10 ==0) and (epoch_i != 0)) or (epoch_i==epochs-1):
            
            print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_out.logits.argmax(dim = -1)])
        
            
        
    torch.save(model.state_dict(), 'BART_classifier_final.pt')
        
    return  train_losses, train_accs, val_losses,val_accs

In [ ]:
stats = train(merge_model, train_dataloader, val_dataloader=None, epochs=epochs, evaluation=False,  config=config)

In [ ]:
torch.save(merge_model.state_dict(), 'BART_classifier_final.pt')

In [ ]:
train_losses, train_accs, val_losses,val_accs = stats

In [ ]:
plt.plot(np.arange(len(train_losses)), np.array(train_losses))

# Testing

b1,b2,b3,b4 = next(iter(train_dataloader))
b1 = b1.cuda()
b2 = b2.cuda()
b3 = b3.cuda()
b4 = b4.cuda()

b1.shape, b2.shape, b3.shape, b4.shape

z1 = torch.nn.Parameter(torch.ones_like(b1).float()).long()
z2 = torch.ones_like(b2)

z1_dash = z1 + 5
z2_dash = z2 + 5

t = summary_model(b1, b2)

t.logits.shape

merge_model.train()

t_dash = merge_model(b1, b2, b3,b4)

t_dash[0].logits.shape

loss = mse_loss(t_dash[1], t_dash[2])

print(torch.autograd.grad(loss,merge_model.summary_model.parameters(), retain_graph=True))